In [3]:
# ⚠️ Requiere: pip install requests
from pathlib import Path
import requests

BASE_URL = "http://localhost:9000"  # ajusta si usas otro puerto/host
PDFS = [
    "FEV_800099860_FEHP755052.pdf",
    "HAM_800099860_FEHP755052.pdf",
    "HAU_800099860_FEHP755052.pdf",
    "HEV_800099860_FEHP755052.pdf",
]

# Enviar como multipart/form-data con el mismo nombre de campo: files
files = [("files", (Path(p).name, Path(p).read_bytes(), "application/pdf")) for p in PDFS]

r = requests.post(f"{BASE_URL}/merge-pdf", files=files, headers={"Accept": "application/pdf"}, timeout=300)
print("HTTP", r.status_code, "| Páginas unidas:", r.headers.get("X-Merged-Pages"))

# Guardar el PDF resultante
out_path = "merged_local.pdf"
Path(out_path).write_bytes(r.content)
print("Guardado:", out_path)


HTTP 200 | Páginas unidas: 16
Guardado: merged_local.pdf


In [2]:
# test_merge_json.py
from pathlib import Path
import base64
import requests

BASE_URL = "http://localhost:9000"   # ajusta si usas otro host/puerto
OUT_FILE = "merged_from_json.pdf"

PDFS = [
    "FEV_800099860_FEHP755052.pdf",
    "HAM_800099860_FEHP755052.pdf",
    "HAU_800099860_FEHP755052.pdf",
    "HEV_800099860_FEHP755052.pdf",
]

files = []
for p in PDFS:
    b = Path(p).read_bytes()
    files.append({
        "name": Path(p).name,
        "data_b64": base64.b64encode(b).decode("ascii"),
        "mime_type": "application/pdf",
    })

payload = {"files": files}

print(f"→ POST {BASE_URL}/merge-pdf-json  ({len(files)} PDFs)")
r = requests.post(f"{BASE_URL}/merge-pdf-json", json=payload, timeout=300)
print("HTTP", r.status_code)

if r.ok:
    Path(OUT_FILE).write_bytes(r.content)
    print("✓ Guardado:", OUT_FILE)
    print("✓ Páginas unidas:", r.headers.get("X-Merged-Pages"))
else:
    print("ERROR:\n", r.text[:2000])


→ POST http://localhost:9000/merge-pdf-json  (4 PDFs)
HTTP 200
✓ Guardado: merged_from_json.pdf
✓ Páginas unidas: 16
